# Description de MNIST "Handwritten Digit Recognition Problem"

MNIST est un jeu de données développé par Yann LeCun, Corinna Cortes et Christopher Burges pour évaluer les modèles d’apprentissage automatique sur la classification des chiffres manuscrits.

Ce jeu de données a été construit à partir de plusieurs ensembles de documents numérisés provenant du National Institute of Standards and Technology (NIST). Nommé **Modified NIST**, ou **MNIST**.

Chaque image est un carré de **28×28 pixels** (soit **784 pixels** au total). Une division standard du jeu de données est utilisée pour l'évaluation et la comparaison des modèles : **60 000 images** servent à l'entraînement du modèle, et un ensemble distinct de **10 000 images** est utilisé pour le tester.

Il s'agit d'une tâche de reconnaissance de chiffres. Il y a donc **dix chiffres** (de 0 à 9), soit **dix classes** à prédire. 

Voici le schéma d’architecture des réseaux de neurones que nous allons réaliser dans ce workshop. 

<img src="TP-CNN-architecture-1.jpg" width="900">

Importe les bibliothèques nécessaires pour :

* Construire le modèle de réseau de neurones (tensorflow, keras)

* Manipuler les données (numpy)

* Visualiser des images et des résultats (matplotlib)

* Organiser les fichiers si nécessaire (os)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os

# 1. Charger la dataset MNIST

La cellule suivante permet de charger le jeu de données MNIST en deux ensembles : x_train, y_train pour l'entraînement, et x_test, y_test pour le test.
L’instruction x_train[0].shape permet de vérifier que chaque image est bien de taille 28x28.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train[0].shape

# 2. Data processing

## 2.1. Normalisation des images (mise à l'échelle entre 0 et 1)

L'objectif est de rendre les valeurs des pixels plus petites (entre 0 et 1) pour faciliter et accélérer l’apprentissage du modèle.

Les images de MNIST ont des valeurs entre 0 et 255. En divisant par 255, on obtient des valeurs entre 0 et 1, ce qui améliore la convergence du modèle pendant l’apprentissage.

In [3]:
x_train, x_test = x_train / 255.0, x_test / 255.0

# 3. Construire les modèles

Introduction à la section dans laquelle on va définir l’architecture du CNN, c’est-à-dire la séquence de couches qui va transformer l’image d’entrée en une prédiction de chiffre.

Comme annoncé en début du workshop nous allons construire trois architectures de réseaux de neurones convolutifs (CNN) de tailles croissantes, pour effectuer une classification sur les images de la dataset MNIST.

In [4]:
def build_small_model():
    # create model
    model = Sequential()
    model.add(keras.layers.Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation='relu'))
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(10, activation='softmax'))
    return model

def build_medium_model():
    # create model
    model = Sequential()
    model.add(keras.layers.Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    return model

def build_large_model():
    # create model
    model = Sequential()
    model.add(keras.layers.Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Conv2D(15, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    return model

# 4. Entraîner les modèles

Nous allons maintenant comparer les différentes architectures CNN (small, medium, large) :

* les entraîner sur MNIST,

* enregistrer leur historique d'apprentissage,

* sauvegarder les modèles pour une réutilisation ultérieure

In [ ]:
# Défnir un dictionnaire de modèles CNN
models = {
    "small": build_small_model(), 
    "medium": build_medium_model(), 
    "large": build_large_model()
    }

#  Définit un dossier où seront enregistrés les résultats.
SAVE_DIR = "mnist_results"
os.makedirs(SAVE_DIR, exist_ok=True)

# Initialiser un dictionnaire pour stocker les historiques d’entraînement (perte et précision) de chaque modèle.
histories = {}

for name, model in models.items():
    model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
    histories[name] = history.history
    model.save(os.path.join(SAVE_DIR, f"{name}_model.h5"))
    np.save(os.path.join(SAVE_DIR, f"{name}_history.npy"), history.history)

<blockquote>
Un optimizer est un algorithme qui ajuste les poids du réseau pendant l'entraînement.

Exemples connus : 'SGD', 'Adam', 'RMSprop', etc.

Vous pouvez essayer différents optimiseurs en les passant au modèle via model.compile(...).

Cette exploration permet de tester l'impact de chaque optimizer sur les performances du modèle.
</blockquote>

In [ ]:
optimizers = [opt for opt in dir(tf.keras.optimizers) if not opt.startswith("__")]
print(optimizers)

# 5. Validation

## 5.1 Plot Accuracy & Loss

In [ ]:
from utils import plot_all_metrics

plot_all_metrics(SAVE_DIR, histories)

## 5.2 Evaluation

On crée un dictionnaire vide qui contiendra les résultats d’évaluation (précision et taux d’erreur) pour chaque modèle (small, medium, large).

Pour cela nous allons parcourir chaque paire (nom, modèle) dans le dictionnaire models. Cela permet de traiter chaque modèle individuellement.

On stocke les résultats d’évaluation dans le dictionnaire results, en convertissant la précision en pourcentage (accuracy * 100) et en calculant le taux d’erreur (100 - accuracy).

In [ ]:
results = {}
for name, model in models.items():
    # Evaluer le modèle sur les données de test (x_test, y_test). NB: Le paramètre verbose=0 désactive l'affichage.
    scores = model.evaluate(x_test, y_test, verbose=0)
    results[name] = {"accuracy": scores[1] * 100, "error_rate": 100 - scores[1] * 100}

print(results)

## 5.3 Exemple de prédiction

Nous allons visualiser un exemple concret de prédiction faite par le modèle, pour comprendre ses performances de manière plus intuitive.

In [ ]:
# Affichage d'une prédiction
def plot_sample_prediction(index):
    plt.imshow(x_test[index].reshape(28, 28), cmap='gray')
    pred = np.argmax(model.predict(x_test[index].reshape(1, 28, 28, 1)))
    plt.title(f"Prédiction: {pred} | Vérité: {y_test[index]}")
    plt.show()

plot_sample_prediction(0)